In [ ]:
import os.path
import ROOT
import uproot
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import matplotlib.ticker as ticker
import os
import pandas as pd

In [ ]:
def extractRMSForResiduals(hist, quantile=0.5, plot=False):
    # Extract bin contents (residuals) and bin edges
    bin_contents = hist.values()
    bin_edges = hist.axis().edges()

    # Reconstruct the distribution
    # To avoid type issues, use list comprehension and avoid np.repeat
    values = np.concatenate([
        np.full(int(count), (bin_edges[i] + bin_edges[i + 1]) / 2)
        for i, count in enumerate(bin_contents)
    ])

    # Determine the quantiles    with open(config_file, 'r') as file:
    lower_quantile = quantile
    upper_quantile = 100 - quantile
    lower_q_value = np.percentile(values, lower_quantile)
    upper_q_value = np.percentile(values, upper_quantile)

    # Filter residuals within the quantiles
    truncated_values = values[(values >= lower_q_value) & (values <= upper_q_value)]
    # Calculate RMS
    truncated_rms = np.std(truncated_values)

    N = np.sum(bin_contents)

    std_err = truncated_rms / np.sqrt(2* (N - 1))

    if plot:

        print(f'lower q ={lower_q_value} for {lower_quantile}th Percentile')
        print(f'upper q ={upper_q_value} for {upper_quantile}th Percetnile')
        print(f'Truncated RMS: {truncated_rms}')
        # Plot the histogram and quantiles
        plt.figure(figsize=(10, 10))
        plt.hist(values, bins=bin_edges, alpha=0.7, label='Residuals', edgecolor='black', color='black')

        # Add lines for quantiles
        plt.axvline(x=lower_q_value, color='r', linestyle='--', linewidth=2, label=f'{lower_quantile}th Percentile')
        plt.axvline(x=upper_q_value, color='g', linestyle='--', linewidth=2, label=f'{upper_quantile}th Percentile')

        plt.xlim((lower_q_value * 2, upper_q_value * 2))
        plt.title(f'Spatial residuals')
        plt.xlabel(r'$x_{Track} - x_{DUT}$ [$\mu m$]')
        plt.ylabel('# Events')
        plt.legend()
        # plt.xlim(xlims)
        plt.grid(True)
        # plt.savefig(output_plot)
        plt.show()
    return  truncated_rms, std_err

In [ ]:
def extractEfficiency(root_file, key):
    file = ROOT.TFile(root_file, 'READ')
    efficiency = file.Get(key)

    if not efficiency:
        print("TEfficiency object not found in the file!")
        return None

    # Access the properties of the TEfficiency object
    if isinstance(efficiency, ROOT.TEfficiency):
            i = 1 # Corry stores effi in first bin
            eff_value = efficiency.GetEfficiency(i)
            eff_error_low = efficiency.GetEfficiencyErrorLow(i)
            eff_error_up = efficiency.GetEfficiencyErrorUp(i)

            # print(f"Bin {i}: Efficiency = {eff_value}, Error Low = {eff_error_low}, Error Up = {eff_error_up}")
    else:
        print("The object is not a TEfficiency object.")
        return  None

    # Close the file
    file.Close()
    return eff_value, eff_error_low, eff_error_up

In [ ]:
# Define the transformation function
def primary_to_secondary(x):
    coeffs = config["polynomial_coefficients"]
    return np.polyval(coeffs, np.asarray(x))  # Ensure input is an array

# Define an approximate inverse function
def secondary_to_primary(x2):
    coeffs = config["polynomial_coefficients"]

    if len(coeffs) == 2:  # If it's a linear function, invert directly
        a, b = coeffs
        return (np.asarray(x2) - b) / a

    x_values = np.linspace(0, 10, 1000)
    y_values = np.polyval(coeffs, x_values)
    return np.interp(x2, y_values, x_values)  # Interpolate inverse

In [ ]:
def init(config):    
    fig = None
    figsize = config.get('figsize', None)
    fontsize = config.get('fontsize', None)
    if figsize and len(figsize) == 2:
        fig = plt.figure(figsize=(figsize[0], figsize[1]))


    if fontsize is not None:
        plt.rcParams.update({
            'font.size': fontsize,  # Base font size
            'axes.titlesize': fontsize,  # Title size
            'axes.labelsize': fontsize,  # X/Y label size
            'xtick.labelsize': fontsize,  # X tick label size
            'ytick.labelsize': fontsize,  # Y tick label size
            'legend.fontsize': fontsize,  # Legend text size
            'legend.title_fontsize': fontsize  # Legend title size
        })

    return fig

In [ ]:
def finish(config, axes=None):
    if axes and not isinstance(axes, list):
        axes = [axes]
    elif not axes:
        axes = [plt.gca()]
    for ax in axes:
        ax.grid(True)
        if config.get('log_x', False):
            ax.set_xscale('log')
            ax.xaxis.set_major_formatter(ticker.LogFormatterSciNotation())
        if config.get('log_y', False):
            ax.set_yscale('log')
            ax.yaxis.set_major_formatter(ticker.LogFormatterSciNotation())

        ax.set_xlabel(config['x_name'])

        xlim = config.get('xlim', None)
        if xlim:
            ax.set_xlim(xlim)
        ylim = config.get('ylim', None)
        if ylim:
            ax.set_ylim(ylim)

        if config.get('show_legend', True):
            # Default to 'best' if not specified
            legend_loc = config.get('legend_loc', 'best')
            ax.legend(loc=legend_loc)
        else:
            ax.legend().remove()

        if config.get('tick_padding', None):
            ax.tick_params(axis='x', pad=config['tick_padding'])
            ax.tick_params(axis='y', pad=config['tick_padding'])

    output_img = config.get('output_img', None)
    if output_img:
        plt.savefig(config['output_img'], bbox_inches='tight')
        print('saved to ', config['output_img'])

In [ ]:
def stdStats(tkey):
    hist_np = tkey.to_numpy()
    #hist_np[0] contains the counts and hist_np[1] the edges
    counts = hist_np[0]
    edges = hist_np[1]


    # bins from uproot are jagged, this means eg if we have 10 bins in a range from 0-10 we just got the bins edges as:
    # [0, 1, 2, 3, ..., 9, 10] -> we have one bin too many
    # for weighted mean calculation we have to extract the mid point, with the example above, we want:
    # [0.5, 1.5, 2.5, ..., 8.5, 9.5]
    # we call this transformation unjagging:
    unjagged_bins = (edges[:-1] + edges[1:]) / 2
    
    N = np.sum(counts)
    mean = np.sum(counts * unjagged_bins) / N    
    std_dev = np.sqrt(np.sum(counts * (unjagged_bins - mean) ** 2) / N)
    std_err = std_dev / np.sqrt(N)

    return mean, std_err, std_dev



In [ ]:
def generate_df(config):
    data = {'X': [], 'Y': [], 'ErrHi': [], 'ErrLo': [], 'Input': [], 'Sample': [], 'Quantity':[], 'Marker': [], 'Color': [], 'Linestyle': []}
    for input in config['inputs']:    
        path_prefix = config.get('path_prefix', None)
        path = input['path'] if not path_prefix else f'{path_prefix}/{input["path"]}'
        root_files = sorted(glob.glob(f'{path}/*.root'))
        plots = config['plots']
        x_regex = config['x_regex']    

        for root_file in root_files:
            with uproot.open(root_file) as file:
                xVal = float(re.search(x_regex, os.path.basename(root_file)).group(1))
                for plot_info in plots:
                    #allow for 'for' loop if just one key is given:
                    keys = plot_info['keys'] if isinstance(plot_info['keys'], list) else [plot_info['keys']]
                    
                    for i, key in enumerate(keys):
                        tkey = None
                        mean_val = None
                        std_dev_val = None
                        error_up = None
                        error_low = None
                        quantity = plot_info['names'][i]

                        done = False
                        N = 0
                        try:
                            if 'eTotalEfficiency' in key:
                                eff_value, error_low, error_up = extractEfficiency(root_file, key)
                                mean_val = eff_value
                                done = True
                            else:
                                tkey = file[key]
                            
                        except Exception as e:
                            print(f'{root_file}: {key} -> {e}')
                            continue 

                        if 'residuals' in key:
                            mean_val, std_err = extractRMSForResiduals(tkey, plot=False)
                        elif isinstance(tkey, uproot.behaviors.TH1.TH1):
                            mean_val, std_err, std_dev_val = stdStats(tkey)
                            
                        elif not done:
                            print("don't know how to handle tkey", tkey)
                            continue   

                        if not error_up and not error_low:
                            error_up = error_low = std_err

                        yscale = plot_info.get('yscale', None)
                        if yscale:
                            mean_val *= yscale
                            error_up *= yscale
                            error_low *= yscale

                        data['X'].append(xVal)
                        data['Y'].append(mean_val)
                        data['ErrHi'].append(error_up)
                        data['ErrLo'].append(error_low)
                        data['Input'].append(input)
                        data['Quantity'].append(quantity)
                        data['Sample'].append(input['name'])
                        data['Marker'].append(input.get('marker'))
                        data['Color'].append(input.get('color'))
                        data['Linestyle'].append(input.get('linestyle'))

# print({k: len(v) for k, v in data.items()})
    return pd.DataFrame(data)
# sns.lineplot(data=df, x='X', y='Y', hue='Sample', style='Quantity')
# df


In [ ]:
def plot_std(df, config, ax = None):
    marker_mapping = {k: v for k, v in zip(df['Sample'], df['Marker']) if pd.notna(v)}
    color_mapping = {k: v for k, v in zip(df['Sample'], df['Color']) if pd.notna(v)}
    linestyle_mapping = {k: v for k, v in zip(df['Sample'], df['Linestyle']) if pd.notna(v)}

    # Ensure that all samples have a mapping, if not, set to None to use default behavior
    all_samples = df['Sample'].unique()
    marker_mapping = True if any(sample not in marker_mapping for sample in all_samples) else marker_mapping
    color_mapping = None if any(sample not in color_mapping for sample in all_samples) else color_mapping
    linestyle_mapping = None if any(sample not in linestyle_mapping for sample in all_samples) else linestyle_mapping
    
    if linestyle_mapping is not None:
        linestyle_mapping = {
            k: (None, None) if v == '-' else (4, 2) if v == '--' else (1, 1)  # Adjust for more styles if needed
            for k, v in linestyle_mapping.items()
    }
    else:
        linestyle_mapping = True

    
    line_width = config.get('line_width', None)

    ax = sns.lineplot(
        data=df, 
        x='X', 
        y='Y', 
        hue='Sample', 
        style='Sample', 
        markers=marker_mapping, 
        dashes=linestyle_mapping, 
        markersize=config.get('markersize', None), 
        palette=color_mapping,
        linewidth=line_width,
        ax=ax
    )    
    if config.get('plot_errorbar', False):
        for sample in df['Sample'].unique():
            sample_df = df[df['Sample'] == sample]
            ax.errorbar(sample_df['X'], sample_df['Y'], yerr=[sample_df['ErrLo'], sample_df['ErrHi']], 
                        fmt='.', color=ax.get_lines()[df['Sample'].unique().tolist().index(sample)].get_color(), capsize=5, linewidth=line_width / 2.0 if line_width else None)
        
    quantities = df['Quantity'].unique()
    if len(quantities) == 1:
        ax.set_ylabel(quantities[0])
    else:
        ax.set_ylabel('Value')
    
    
    # # Apply custom linestyles
    # for line, sample in zip(ax.get_lines(), df['Sample'].unique()):
    #     if linestyle_mapping is not None and sample in linestyle_mapping:
    #         line.set_linestyle(linestyle_mapping[sample])
    
    # # Update legend to reflect custom linestyles
    # handles, labels = ax.get_legend_handles_labels()
    # for handle, label in zip(handles, labels):
    #     if linestyle_mapping is not None and label in linestyle_mapping:
    #         handle.set_linestyle(linestyle_mapping[label])
    # ax.legend(handles=handles, labels=labels)

    if config.get('do_annotate', False):
        # Get line colors from the plot and map to Sample name
        line_colors = {line.get_label(): line.get_color() for line in ax.get_lines()}
        for i, row in df.iterrows():
            sample_color = line_colors[row["Sample"]]
            ax.text(row["X"], row["Y"] + 0.1, f"{row['Y']:.1f}", 
                    color=sample_color, ha="center")
            
        
    return ax

            
    

In [ ]:
def evaluateSpatialResolution(quantity, dfBiased, dfUnbiased):
    # Filter the DataFrames by the quantity
    dfBiased = dfBiased[dfBiased['Quantity'] == quantity]
    dfUnbiased = dfUnbiased[dfUnbiased['Quantity'] == quantity]

    # Create a new DataFrame for the results
    result_df = pd.DataFrame()

    # Calculate the geometric mean and its error
    result_df['X'] = dfBiased['X']
    result_df['Y'] = np.sqrt(dfBiased['Y'] * dfUnbiased['Y'])
    result_df['ErrHi'] = result_df['Y'] * np.sqrt(
        (dfBiased['ErrHi'] / dfBiased['Y'])**2 + (dfUnbiased['ErrHi'] / dfUnbiased['Y'])**2
    )
    result_df['ErrLo'] = result_df['ErrHi']  # Assuming symmetric errors

    # Copy the other columns from the biased DataFrame
    result_df['Input'] = dfBiased['Input']
    result_df['Sample'] = dfBiased['Sample']
    result_df['Quantity'] = dfBiased['Quantity']
    result_df['Marker'] = dfBiased['Marker']
    result_df['Color'] = dfBiased['Color']
    result_df['Linestyle'] = dfBiased['Linestyle']

    return result_df


In [ ]:
config_unbiased = {
    'path_prefix': '/home/bernhard/cernbox/Diss/mpw4/tb/desy_oct24/analysis',
    "inputs": [
        {
            "path": "1E14Backside/hv",
            "name": "1E14 Backside"
        },
        {
            "path": "1E14Topside/hv",
            "name": "1E14 Topside"
        },
        {
            "path": "3E14Backside/hv",
            "name": "3E14 Backside"
        },
        {
            "path": "3E14Topside/hv",
            "name": "3E14 Topside"
        },
        {
            "path": "1E15_masked/hv",
            "name": "1E15 Topside"
        }
    ],
    "plots": [
        {
            "keys": ["AnalysisDUT/RD50_MPWx_0/local_residuals/residualsX"],
            "names": ["ResidualsX"],
            "yscale": 1,
        }
    ],
    "x_name": "Reverse bias voltage [V]",
    "x_regex": r"hv-(\d+)_",
}

dfUnbiased = generate_df(config_unbiased)

config_biased = {
    'path_prefix': '/home/bernhard/cernbox/Diss/mpw4/tb/desy_oct24/analysis',
    "inputs": [
        {
            "path": "1E14Backside/hv-biasedRes",
            "name": "1E14 Backside"
        },
        {
            "path": "1E14Topside/hv-biasedRes",
            "name": "1E14 Topside"
        },
        {
            "path": "3E14Backside/hv-biasedRes",
            "name": "3E14 Backside"
        },
        {
            "path": "3E14Topside/hv-biasedRes",
            "name": "3E14 Topside"
        },
        {
            "path": "1E15_masked/hv-biasedRes",
            "name": "1E15 Topside"
        }
    ],
    "plots": [
        {
            "keys": ["AnalysisDUT/RD50_MPWx_0/local_residuals/residualsX"],
            "names": ["ResidualsX"],
            "yscale": 1,
        }
    ],
    "x_name": "Reverse bias voltage [V]",
    "x_regex": r"hv-(\d+)_",
}
dfBiased = generate_df(config_biased)

In [ ]:
df = evaluateSpatialResolution("ResidualsX", dfBiased, dfUnbiased)
df

In [ ]:
config_unbiased = {
    'path_prefix': '/home/bernhard/cernbox/Diss/mpw4/tb/desy_oct24/analysis',
    "inputs": [
        {
            "path": "../../desy_apr24/analysis/harmonized/V-W3/bias",
            "name": "Non Irradiated - Backside",
            "color": "blue",
            "linestyle": "-"

        },
        {
            "path": "1E14Backside/hv",
            "name": "1E14 Backside",
            "color": "green",
            "linestyle": "-"
        },
        {
            "path": "1E14Topside/hv",
            "name": "1E14-Topside",
            "color": "green",
            "linestyle": "--"            
        },
        {
            "path": "3E14Backside/hv",
            "name": "3E14-Backside",
            "color": "orange",
            "linestyle": "-"            
        },
        {
            "path": "3E14Topside/hv",
            "name": "3E14-Topside",
            "color": "orange",
            "linestyle": "--"
        },
        {
            "path": "1E15_masked/hv",
            "name": "1E15-Topside",
            "color": "red",
            "linestyle": "--"
        }
    ],
    "plots": [
        {
            "keys": ["AnalysisDUT/RD50_MPWx_0/local_residuals/residualsX"],
            "names": ["Spatial Resolution [$\\mu m$]"],
            "yscale": 1,
        }
    ],
    "x_name": "Reverse bias voltage [V]",
    "x_regex": r"(?:_bias|hv).*?(\d+)",
}

dfUnbiased = generate_df(config_unbiased)

config_biased = {
    'path_prefix': '/home/bernhard/cernbox/Diss/mpw4/tb/desy_oct24/analysis',
    "inputs": [
        {
            "path": "../../desy_apr24/analysis/harmonized/V-W3/bias-biasedRes",
            "name": "Non Irradiated",
            "color": "blue",
            "linestyle": "-"
        },
        {
            "path": "1E14Backside/hv-biasedRes",
            "name": "1E14-Backside",
            "color": "green",
            "linestyle": "-"
        },
        {
            "path": "1E14Topside/hv-biasedRes",
            "name": "1E14-Topside",
            "color": "green",
            "linestyle": "--"
        },
        {
            "path": "3E14Backside/hv-biasedRes",
            "name": "3E14-Backside",
            "color": "orange",
            "linestyle": "-"
        },
        {
            "path": "3E14Topside/hv-biasedRes",
            "name": "3E14-Topside",
            "color": "orange",
            "linestyle": "--"
        },
        {
            "path": "1E15_masked/hv-biasedRes",
            "name": "1E15-Topside",
            "color": "red",
            "linestyle": "--"
        }
    ],
    "plots": [
        {
            "keys": ["AnalysisDUT/RD50_MPWx_0/local_residuals/residualsX"],
            "names": ["Spatial Resolution [$\\mu m$]"],
            "yscale": 1,
        }
    ],
    "x_name": "Reverse bias voltage [V]",
    "x_regex": r"(?:_bias|hv).*?(\d+)",
}
dfBiased = generate_df(config_biased)



In [ ]:

df = evaluateSpatialResolution("Spatial Resolution [$\\mu m$]", dfBiased, dfUnbiased)

config_plot = {
    "plots": [
        {
            "keys": ["AnalysisDUT/RD50_MPWx_0/local_residuals/residualsX"],
            # "names": [r"ResidualsX [$\mu m$]"],
            "yscale": 1,
        }
    ],
    "x_name": "Reverse bias voltage [V]",
    "x_regex": r"(?:_bias|hv).*?(\d+)",
    'plot_errorbar': True,
    'do_annotate': False,
    'line_width': 4,
    'output_img': "/home/bernhard/cernbox/Diss/paper/papierln/vci2025/paper/figs/res_vs_bias_irradiated.png",
    'figsize': [25, 15],
    'legend_loc': 'upper left',
}
init(config_plot)
ax = plot_std(df, config_plot)
finish(config_plot)

In [ ]:
config = {
    'path_prefix': '/home/bernhard/cernbox/Diss/mpw4/tb/desy_oct24/analysis',
    "inputs": [
        {
            "path": "../../desy_apr24/analysis/harmonized/V-W3/bias",
            "name": "Non Irradiated-Backside",
            "color": "blue",
            "linestyle": "-"

        },
        {
            "path": "1E14Backside/hv",
            "name": "1E14-Backside",
            "color": "green",
            "linestyle": "-"
        },
        {
            "path": "1E14Topside/hv",
            "name": "1E14-Topside",
            "color": "green",
            "linestyle": "--"            
        },
        {
            "path": "3E14Backside/hv",
            "name": "3E14-Backside",
            "color": "orange",
            "linestyle": "-"            
        },
        {
            "path": "3E14Topside/hv",
            "name": "3E14-Topside",
            "color": "orange",
            "linestyle": "--"
        },
        {
            "path": "1E15_masked/hv",
            "name": "1E15-Topside",
            "color": "red",
            "linestyle": "--"
        }
    ],
    "plots": [
        {
            "keys": ["ClusteringSpatial/RD50_MPWx_0/clusterSize"],
            "names": ["Cluster size"],
            "yscale": 1,
        }
    ],
    "x_name": "Reverse bias voltage [V]",
    "x_regex": r"(?:_bias|hv).*?(\d+)",
    'plot_errorbar': True,
    'do_annotate': False,
    'line_width': 4,
    'output_img': "/home/bernhard/cernbox/Diss/paper/papierln/vci2025/paper/figs/clstrsz_vs_bias.png",
    'figsize': [25, 15],
    'fontsize': 45,
    'legend_loc': 'upper left',
}
init(config)
df = generate_df(config)
ax = plot_std(df, config)
finish(config)

In [ ]:
config['plots'] = [
    {
        "keys": ["EventLoaderEUDAQ2/RD50_MPWx_0/hPixelRawValues"],
        "names": ["Time over Threshold [25ns]"],
        "yscale": 1,
    }
]
config['output_img'] = "/home/bernhard/cernbox/Diss/paper/papierln/vci2025/paper/figs/tot_vs_bias.png"
init(config)
df = generate_df(config)
plot_std(df, config)
finish(config)

In [ ]:
config = {
    'path_prefix': '/home/bernhard/cernbox/Diss/mpw4/tb/desy_oct24/analysis',
    "inputs": [
        {
            "path": "../../desy_apr24/analysis/harmonized/V-W3/bias",
            "name": "Non Irradiated-Backside",
            "color": "blue",
            "linestyle": "-"

        },
        {
            "path": "1E14Backside/hv",
            "name": "1E14-Backside",
            "color": "green",
            "linestyle": "-"
        },
        {
            "path": "1E14Topside/hv",
            "name": "1E14-Topside",
            "color": "green",
            "linestyle": "--"            
        },
        {
            "path": "3E14Backside/hv",
            "name": "3E14-Backside",
            "color": "orange",
            "linestyle": "-"            
        },
        {
            "path": "3E14Topside/hv",
            "name": "3E14-Topside",
            "color": "orange",
            "linestyle": "--"
        },
        {
            "path": "1E15_masked/hv",
            "name": "1E15-Topside",
            "color": "red",
            "linestyle": "--"
        }
    ],
    "plots": [
        {
            "keys": ["AnalysisEfficiency/RD50_MPWx_0/eTotalEfficiency"],
            "names": ["Efficiency [%]"],
            "yscale": 100,
        }
    ],
    "x_name": "Reverse bias voltage [V]",
    "x_regex": r"(?:_bias|hv).*?(\d+)",
    'plot_errorbar': True,
    'do_annotate': False,
    'line_width': 4,
    'output_img': "/home/bernhard/cernbox/Diss/paper/papierln/vci2025/paper/figs/effi_vs_bias_irradiated.png",
    'figsize': [25, 15],
    'ylim': [85, 100],
    'fontsize': 40,
    'legend_loc': 'upper left'
}
init(config)
df = generate_df(config)
ax = plot_std(df, config)

finish(config, ax)

In [ ]:
config = {
    "inputs": [
        {
            "path": "/home/bernhard/cernbox/Diss/mpw4/tb/desy_apr24/analysis/harmonized/tAnalysis/V-W3/thr",
            "name": "In-time Efficiency",

        }
    ],
    "plots": [
        {
            "keys": ["AnalysisEfficiency/RD50_MPWx_0/eTotalEfficiency"],
            "names": ["In-time Efficiency [%]"],
            "yscale": 100,
        }
    ],
    "x_name": "Absolute Time Cut [ns]",
    "x_regex": r"tCut(\d+)ns",
    'plot_errorbar': True,
    'do_annotate': False,
    'line_width': 4,
    'output_img': "/home/bernhard/cernbox/Diss/paper/papierln/vci2025/paper/figs/intimeEffi.png",
    'figsize': [25, 13],
    'ylim': [94, 100.5],
    'show_legend': False,
    'tick_padding': 15,
    'fontsize': 50
}
fig = init(config)
df = generate_df(config)

ax1 = fig.add_subplot(121)  # First subplot
ax2 = fig.add_subplot(122, sharey=ax1)   # Second subplot sharing y-axis

ax1.set_xlim([0, 60])
ax2.set_xlim([6300, 6500])

plot_std(df, config, ax1)
ax2.tick_params(left=False, labelleft=False)  # Remove y-axis ticks and labels
plot_std(df, config, ax2)
finish(config, [ax1, ax2])